In [1]:
import pandas as pd #import pandas library to handle file, database

In [2]:
dataset = pd.read_csv("insurance_pre.csv") #read the file data and store it in dataset variable

In [3]:
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
#data preprocessing - because categorial data can't be handle so we modify the data into numerical data using one hot encoding(Nominal)

dataset = pd.get_dummies(dataset, drop_first=True)

dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,False,True
1,18,33.770,1,1725.55230,True,False
2,28,33.000,3,4449.46200,True,False
3,33,22.705,0,21984.47061,True,False
4,32,28.880,0,3866.85520,True,False
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,True,False
1334,18,31.920,0,2205.98080,False,False
1335,18,36.850,0,1629.83350,False,False
1336,21,25.800,0,2007.94500,False,False


In [9]:
boolean_columns = dataset.select_dtypes(include=['bool']).columns #get boolean type column data from the dataset and store it in boolean_columns variable
print(boolean_columns)

dataset[boolean_columns] = dataset[boolean_columns].astype(int) #convert boolean column dataset to 0's and 1's
print(dataset[boolean_columns])

Index(['sex_male', 'smoker_yes'], dtype='object')
      sex_male  smoker_yes
0            0           1
1            1           0
2            1           0
3            1           0
4            1           0
...        ...         ...
1333         1           0
1334         0           0
1335         0           0
1336         0           0
1337         0           1

[1338 rows x 2 columns]


In [11]:
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [13]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [15]:
#split input data
independent = dataset [['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]

independent

,age,bmi,children,sex_male,smoker_yes
0,19,27.900,0,0,1
1,18,33.770,1,1,0
2,28,33.000,3,1,0
3,33,22.705,0,1,0
4,32,28.880,0,1,0
...,...,...,...,...,...
1333,50,30.970,3,1,0
1334,18,31.920,0,0,0
1335,18,36.850,0,0,0
1336,21,25.800,0,0,0


In [17]:
dependent = dataset[['charges']] #split output data

dependent

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


In [19]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor #Create a model

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.5, 1],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2]
}

grid = GridSearchCV(XGBRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='r2')

grid.fit(independent, dependent)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, feature_weights=None,
                                    gamma=None, grow_policy=None,
                                    importance_type=None,
                                    interaction_constraints=None,
                                    lear...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.5, 1], 'n_estimators': [100, 200],
                         'reg_alpha': [0, 0.1, 1], 'reg_lambda': [1, 1.5, 2]},
             scoring='r2', verbose=3)

In [21]:
re = grid.cv_results_

#print("CV_RESULTS : ", re)

grid_pred = grid.predict(independent) #test the model with input data

from sklearn.metrics import r2_score #Evaluation metrics

r_score = r2_score(dependent, grid_pred)
print("R score value is : ", r_score) 

print("Best Score:", grid.best_score_)
print("Best Parameters:", grid.best_params_)

R score value is :  0.9945890307426453
Best Score: 0.7965347528457641
Best Parameters: {'learning_rate': 0.5, 'n_estimators': 100, 'reg_alpha': 0.1, 'reg_lambda': 2}


In [23]:
import pandas as pd

# feature names 
feature_names = ['age', 'bmi', 'children', 'sex_male', 'smoker_yes']  

independent_df = pd.DataFrame(independent, columns=feature_names)

# Get feature importances from best model inside GridSearchCV
importance_scores = grid.best_estimator_.feature_importances_

# Create DataFrame of feature importances
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importance_scores
})

# Sort by most important feature
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display the result
print(importance_df)


      Feature  Importance
4  smoker_yes    0.948778
1         bmi    0.021545
0         age    0.016403
2    children    0.007811
3    sex_male    0.005463


In [25]:
table = pd.DataFrame.from_dict(re) #convert dictonary format to DataFrame(table)

In [27]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_n_estimators,param_reg_alpha,param_reg_lambda,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.139773,0.008141,0.010923,0.001617,0.5,100,0.0,1.0,"{'learning_rate': 0.5, 'n_estimators': 100, 'r...",0.788922,0.721893,0.813313,0.796220,0.816301,0.787330,0.034287,11
1,0.131653,0.010350,0.013324,0.003110,0.5,100,0.0,1.5,"{'learning_rate': 0.5, 'n_estimators': 100, 'r...",0.802132,0.718009,0.808448,0.799431,0.819843,0.789573,0.036466,8
2,0.125151,0.006831,0.012021,0.002949,0.5,100,0.0,2.0,"{'learning_rate': 0.5, 'n_estimators': 100, 'r...",0.804779,0.719653,0.833720,0.803161,0.821061,0.796475,0.040017,2
3,0.125169,0.005570,0.011265,0.002801,0.5,100,0.1,1.0,"{'learning_rate': 0.5, 'n_estimators': 100, 'r...",0.788923,0.721894,0.811041,0.795867,0.815586,0.786662,0.033810,12
4,0.117304,0.008962,0.007613,0.002969,0.5,100,0.1,1.5,"{'learning_rate': 0.5, 'n_estimators': 100, 'r...",0.806853,0.718010,0.808449,0.799432,0.819843,0.790517,0.036838,6
5,0.115603,0.013057,0.009981,0.002990,0.5,100,0.1,2.0,"{'learning_rate': 0.5, 'n_estimators': 100, 'r...",0.805077,0.719653,0.833720,0.803162,0.821062,0.796535,0.040030,1
6,0.123008,0.009134,0.009306,0.002490,0.5,100,1.0,1.0,"{'learning_rate': 0.5, 'n_estimators': 100, 'r...",0.790929,0.721969,0.811803,0.797506,0.816688,0.787779,0.034200,10
7,0.118780,0.009721,0.010470,0.003149,0.5,100,1.0,1.5,"{'learning_rate': 0.5, 'n_estimators': 100, 'r...",0.806938,0.731293,0.811461,0.787661,0.813351,0.790141,0.030804,7
8,0.122160,0.007105,0.009342,0.004510,0.5,100,1.0,2.0,"{'learning_rate': 0.5, 'n_estimators': 100, 'r...",0.803929,0.720182,0.831288,0.798074,0.819638,0.794622,0.039013,3
9,0.236659,0.023946,0.006574,0.003865,0.5,200,0.0,1.0,"{'learning_rate': 0.5, 'n_estimators': 200, 'r...",0.784451,0.718356,0.809455,0.795734,0.814182,0.784436,0.034655,16


In [29]:
age = int(input("Enter the age:"))
bmi = float(input("Enter the BMI:"))
children = int(input("Enter the children number:"))
sex = int(input("Enter sex (0 for female and 1 for male):"))
smoker = int(input("Enter 0 for non smoker and 1 for smoker:"))

Enter the age: 26
Enter the BMI: 20
Enter the children number: 0
Enter sex (0 for female and 1 for male): 0
Enter 0 for non smoker and 1 for smoker: 1


In [31]:
Final_prediction =grid.predict([[age,bmi,children,sex,smoker]])
print("Final_prediction of XGboost: {}".format(Final_prediction))

Final_prediction of XGboost: [14922.687]


In [33]:
#save the model using pickle library

import pickle
fileName = "XGboost_grid.sav"
pickle.dump(grid, open(fileName, "wb")) #saved model available in regressor variable. Now storing that variable into fileName

In [35]:
loaded_model = pickle.load(open("XGboost_grid.sav","rb"))
result = loaded_model.predict([[30,30,2,1,1]])  #30,25.899,2,1,1

In [37]:
result

array([20002.26], dtype=float32)